In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset
from utils.watchmen import IsolatingWatchman
from utils.metrics import time_span_metrics
from utils.custom_plots import plot_stacked

## GHL

In [15]:
ds = GhlKasperskyDataset()
ds.shake_not_stir(valid_test_ratio=0.3)

In [16]:
watchman = IsolatingWatchman(random_state=31)

In [17]:
train_gen = ds.train_generator()
for train, _, _ in tqdm(train_gen, total=1):
#     print(train.index[-1] - train.index[0])
    watchman.partial_fit(train)

  0%|          | 0/1 [00:00<?, ?it/s]

1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
1 days 00:59:00
0 days 16:40:00


In [7]:
valid_gen = ds.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen, total=19):
    detect = watchman.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)
#     if max(faults) and sum(detect)>10:
#         print(watchman.forest.score_samples(valid))
#         print(min(watchman.forest.score_samples(valid)))
#         print(detect.values)
#         break

#         plot_stacked(data,
#                      title=info,
#                      group='value_unit',
#                      faults=faults,
#                      detect=detect,
#                     )

  0%|          | 0/19 [00:00<?, ?it/s]

In [8]:
examine_list.mean()

precision    0.019365
recall       0.922619
f1_score     0.037797
dtype: float64

## TEP Harvard

In [9]:
ds = TepHarvardDataset()
ds.shake_not_stir(valid_test_ratio=0.3, balanced_test=True)

In [10]:
watchman = IsolatingWatchman(random_state=31)

In [11]:
train_gen = ds.train_generator()
for train, _, _ in tqdm(train_gen, total=500):
    watchman.partial_fit(train)

  0%|          | 0/500 [00:00<?, ?it/s]

In [12]:
valid_gen = ds.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen, total=300):
    detect = watchman.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)
#     if max(faults) and sum(detect)>10:
#         print(watchman.forest.score_samples(valid)<-0.5)
#         print(min(watchman.forest.score_samples(valid)))
#         print(detect.values)
#         break
#     if max(faults) and max(detect):
#         plot_stacked(data,
#                      title=info,
#                      group='value_unit',
#                      faults=faults,
#                      detect=detect,
#                     )

  0%|          | 0/300 [00:00<?, ?it/s]

In [13]:
examine_list.mean()

precision    0.843261
recall       0.446354
f1_score     0.512470
dtype: float64

In [14]:
stop

NameError: name 'stop' is not defined

## TEP Kaspersky

In [ ]:
ds = TepKasperskyDataset()
ds.shake_not_stir(valid_test_ratio=0.3)

In [ ]:
watchman = IsolatingWatchman(random_state=31)

In [ ]:
watchman.forest

In [ ]:
train_gen = ds.train_generator()
for train, _, _ in tqdm(train_gen, total=400):
    watchman.partial_fit(train)

In [ ]:
watchman

In [ ]:
valid_gen = ds.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen, total=115):
    detect = watchman.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)
#     if max(faults) and max(detect):
#         plot_stacked(data,
#                      title=info,
#                      group='value_unit',
#                      faults=faults,
#                      detect=detect,
#                     )

In [ ]:
examine_list.mean()